In [2]:
import pandas as pd
import numpy as np
import re
import string

## Pre Processing

In [3]:
# Load IMDB dataset from CSV
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch

C:\Users\sdidd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv("../csv/Preprocessed_data.csv")

In [35]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# Take a subset of 5000 rows
subset_df = df.sample(n=50, random_state=42)

In [37]:
# Function to clean and preprocess text
def preprocess_text(text):
    # Remove HTML tags using BeautifulSoup
    text = BeautifulSoup(text, 'html.parser').get_text()

    # Remove punctuation using regular expressions
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Convert to lowercase
    text = text.lower()

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Join the tokens back into a single string
    processed_text = ' '.join(tokens)

    return processed_text

# Apply preprocessing to the 'text' column
subset_df['processed_text'] = subset_df['OriginalReviews'].apply(preprocess_text)

In [38]:

# Save the subset to a new CSV file
subset_df.to_csv('../Dataset/subset_dataset.csv', index=False)

# Assuming your CSV has columns 'review' and 'sentiment'
texts = subset_df['OriginalReviews'].values.tolist()
labels = subset_df['OutputSentiment'].values.tolist()

In [41]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split your data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Map labels to integers
label_mapping = {'positive': 1, 'negative': 0}
train_labels = [label_mapping[label] for label in train_labels]
val_labels = [label_mapping[label] for label in val_labels]

# Load BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)  # Assuming binary classification

# Tokenize and encode the training and validation data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256, return_tensors='tf')
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256, return_tensors='tf')

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': train_encodings['input_ids'], 'attention_mask': train_encodings['attention_mask']},
    tf.constant(train_labels)
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': val_encodings['input_ids'], 'attention_mask': val_encodings['attention_mask']},
    tf.constant(val_labels)
))

# Set up training parameters
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

# Compile the model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=[metric])

# Train the model
history = model.fit(train_dataset.shuffle(1000).batch(8), epochs=3, validation_data=val_dataset.batch(8))

# Evaluate the model on the validation set
val_predictions = model.predict(val_dataset.batch(8))
val_predictions = tf.argmax(val_predictions.logits, axis=1)
val_accuracy = accuracy_score(val_labels, val_predictions)

print(f"Validation Accuracy: {val_accuracy}")

# Save the trained model
model.save_pretrained("bert_sentiment_model")
tokenizer.save_pretrained("bert_sentiment_model")


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
5/5 [==============================] - 70s 9s/step - loss: 0.7390 - accuracy: 0.4500 - val_loss: 0.6771 - val_accuracy: 0.6000
Epoch 2/3
5/5 [==============================] - 41s 8s/step - loss: 0.6925 - accuracy: 0.4750 - val_loss: 0.6836 - val_accuracy: 0.5000
Epoch 3/3
2/2 [==============================] - 6s 390ms/step
Validation Accuracy: 0.4


('bert_sentiment_model\\tokenizer_config.json',
 'bert_sentiment_model\\special_tokens_map.json',
 'bert_sentiment_model\\vocab.txt',
 'bert_sentiment_model\\added_tokens.json',
 'bert_sentiment_model\\tokenizer.json')